<a href="https://colab.research.google.com/github/Aakash326/ML-models/blob/main/personal_assitant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-google-genai langchain-community langchain-chroma
!pip install -q chromadb sentence-transformers
!pip install -q python-docx PyPDF2 openpyxl pypdf
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 116.7 MB/s eta 0:00:0

In [2]:
!pip install -q python-docx PyPDF2 openpyxl pypdf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import glob
import shutil
from pathlib import Path
import gradio as gr
from datetime import datetime

# Document processing
from langchain.document_loaders import (
    DirectoryLoader,
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader,
    CSVLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Embeddings and Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# LLM and Chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [5]:
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("✅ API key loaded successfully")
except:
    print("⚠️  Please add your GOOGLE_API_KEY to Colab Secrets")
    GOOGLE_API_KEY = input("Enter your Google API key: ")
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY


✅ API key loaded successfully


In [6]:
def find_all(drive_path="/content/drive", max_files=2000):  # Changed from 1000 to 2000
    print(f"Scanning all {drive_path} for files")
    supported_extensions = {
        '.txt', '.md', '.py', '.js', '.html', '.css', '.json', '.xml',
        '.pdf', '.docx', '.doc', '.csv', '.xlsx', '.xls','.ipynb'
    }

    all_files = []
    try:
        for root, dirs, files in os.walk(drive_path):
            dirs[:] = [d for d in dirs if not d.startswith('.') and d not in [
                'trash', '.shortcut-targets-by-ids'
            ]]

            for file in files:
                if len(all_files) >= max_files:
                    break

                file_path = os.path.join(root, file)
                file_ext = Path(file).suffix.lower()

                if file_ext in supported_extensions:
                    # Skip very large files (>50MB)
                    try:
                        file_size = os.path.getsize(file_path)
                        if file_size > 50 * 1024 * 1024:  # 50MB limit
                            print(f"Skipping large file: {file} ({file_size / (1024*1024):.1f}MB)")
                            continue
                    except:
                        continue

                    all_files.append(file_path)

                if len(all_files) >= max_files:
                    break

    except Exception as e:
        print(f"An error occurred: {e}")

    return all_files

In [7]:
files=find_all(drive_path="/content/drive")

Scanning all /content/drive for files
Skipping large file: aakash.pdf (135.8MB)


In [8]:
def load_documents(file_path):
    try:
        file_ext = Path(file_path).suffix.lower()

        if file_ext == '.pdf':
            loader = PyPDFLoader(file_path)
        elif file_ext in ['.docx', '.doc']:  # Fixed: missing quote
            loader = Docx2txtLoader(file_path)
        elif file_ext == '.csv':
            loader = CSVLoader(file_path)
        elif file_ext in ['.txt', '.md', '.py', '.js', '.html', '.css', '.json', '.xml']:  # Fixed: missing closing bracket
            loader = TextLoader(file_path, encoding='utf-8')
        else:
            return None

        docs = loader.load()

        for doc in docs:
            doc.metadata.update({
                'file_path': file_path,  # Fixed: hyphen -> underscore
                'file_name': os.path.basename(file_path),
                'file_type': file_ext[1:],
                'file_size': os.path.getsize(file_path) if os.path.exists(file_path) else 0  # Fixed: missing default value
            })

        return docs

    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [9]:
def create_base(drive_path="/content/drive", db_name="personal_db", max_files=2000):  # Changed from 500 to 2000
    print("Scanning drive for files...")
    all_files = find_all(drive_path, max_files)

    if not all_files:
        print("No files found. Exiting...")
        return None

    print(f"Found {len(all_files)} files to process")

    all_documents = []
    successful_load = 0
    failed_load = 0

    for i, file_path in enumerate(all_files):
        if i % 10 == 0:
            print(f"Processing file {i+1}/{len(all_files)}")

        docs = load_documents(file_path)
        if docs:
            all_documents.extend(docs)
            successful_load += 1
        else:
            failed_load += 1

    print(f"✅ Successfully loaded: {successful_load} files")
    print(f"❌ Failed to load: {failed_load} files")
    print(f"📄 Total documents: {len(all_documents)}")

    if not all_documents:
        print("No documents could be loaded. Exiting...")
        return None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    print("Splitting documents into chunks...")
    chunks = text_splitter.split_documents(all_documents)
    print(f"📝 Total chunks: {len(chunks)}")

    print("Creating embeddings...")
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if os.path.exists(db_name):
        shutil.rmtree(db_name)

    print("Creating vector database...")
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory=db_name
    )

    print(f"🎉 Knowledge base created successfully with {len(chunks)} chunks")
    return vectorstore

In [ ]:
def cleanup_and_retry():
    """Clean up and retry database creation"""
    print("🧹 Cleaning up previous attempts...")

    # Remove any existing databases
    for db_path in ["personal_db", "/tmp/personal_db*"]:
        if os.path.exists(db_path):
            try:
                shutil.rmtree(db_path)
                print(f"Removed: {db_path}")
            except:
                pass

    # Clear any ChromaDB cache
    try:
        import chromadb
        # Reset ChromaDB client
        chromadb.reset()
        print("ChromaDB cache cleared")
    except:
        pass

    print("✅ Cleanup complete. Now retry creating the database.")

In [11]:
cleanup_and_retry()

NameError: name 'cleanup_and_retry' is not defined

In [10]:
base=create_base(drive_path="/content/drive", db_name="personal_db", max_files=2000)

Scanning drive for files...
Scanning all /content/drive for files
Skipping large file: aakash.pdf (135.8MB)
Found 209 files to process
Processing file 1/209
Processing file 11/209
Processing file 21/209
Error loading /content/drive/MyDrive/Imp Ques.docx: No module named 'docx2txt'
Processing file 31/209


Error loading /content/drive/MyDrive/0622 SE assignment 1 (1).docx: No module named 'docx2txt'
Error loading /content/drive/MyDrive/1437 SE assignment 1 (1).docx: No module named 'docx2txt'
Processing file 41/209
Error loading /content/drive/MyDrive/HU22CSEN0101437.docx: No module named 'docx2txt'
Error loading /content/drive/MyDrive/WAD CASESTUDY.docx: No module named 'docx2txt'


Processing file 51/209
Processing file 61/209
Processing file 71/209
Processing file 81/209
Processing file 91/209


Processing file 101/209
Processing file 111/209
Processing file 121/209
Processing file 131/209
Processing file 141/209
Processing file 151/209
Processing file 161/209


Error loading /content/drive/MyDrive/Heyehshehs/EAadhaar_2081121746942420211218125647_29082022213843.pdf: File has not been decrypted
Processing file 171/209
Processing file 181/209
Processing file 191/209
Processing file 201/209
✅ Successfully loaded: 62 files
❌ Failed to load: 147 files
📄 Total documents: 2543
Splitting documents into chunks...
📝 Total chunks: 2309
Creating embeddings...


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating vector database...
🎉 Knowledge base created successfully with 2309 chunks


In [12]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI

def create_personal_agent(base):
    """
    Create a personal RAG agent using Gemini and a document vectorstore.
    """

    # 🧠 Custom system prompt
    personal_prompt = PromptTemplate(
        template="""You are my personal AI assistant with access to all my files and documents.

INSTRUCTIONS:
- Answer questions based on the content from my personal files
- Provide detailed, helpful responses (minimum 3-4 sentences)
- If you can't find the answer in my files, say "I couldn't find this information in your files"
- Reference the specific files or documents when relevant
- Be conversational and helpful, like a knowledgeable personal assistant

My Files Content:
{context}

My Question: {question}

Your Response:""",
        input_variables=["context", "question"]
    )

    # 🤖 Gemini LLM via LangChain wrapper
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.7,
        max_output_tokens=1000
    )

    # 🧠 Conversation memory to track previous messages
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    # 🔍 RAG Retriever
    retriever = base.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 20,
            "fetch_k": 40,
            "lambda_mult": 0.7
        }
    )

    # 🔗 Conversational RAG chain
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": personal_prompt},
        return_source_documents=True,
        verbose=True
    )

    return conversation_chain


In [13]:
chain=create_personal_agent(base)

/tmp/ipython-input-12-2052281399.py:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [14]:
def personal_chat(message, history, chain):
    """
    Enhanced chat function for personal assistant
    """
    try:
        result = chain.invoke({"question": message})
        answer = result["answer"]

        # Add source information
        source_docs = result.get('source_documents', [])
        if source_docs:
            unique_files = set()
            for doc in source_docs[:5]:  # Show up to 5 source files
                file_name = doc.metadata.get('file_name', 'Unknown')
                unique_files.add(file_name)

            if unique_files:
                source_info = f"\n\n📁 Sources: {', '.join(list(unique_files)[:3])}"
                if len(unique_files) > 3:
                    source_info += f" + {len(unique_files) - 3} more files"
                answer += source_info

        return answer

    except Exception as e:
        return f"❌ Error: {str(e)}"

In [15]:
def main():
    """
    Main function to set up everything
    """
    print("🎯 Setting up your personal RAG agent...")

    # Create knowledge base
    vectorstore = base

    if not vectorstore:
        print("❌ Failed to create knowledge base")
        return

    # Create agent
    agent = chain

    # Create chat interface
    def chat_interface(message, history):
        return personal_chat(message, history, agent)

    # Launch Gradio
    print("🚀 Launching your personal assistant...")

    interface = gr.ChatInterface(
        chat_interface,
        type="messages",
        title="🤖 My Personal AI Assistant",
        description="Ask me anything about your files and documents!",
        theme=gr.themes.Soft(),
        fill_height=True
    )

    interface.launch(share=True, inbrowser=True)

In [16]:
main()

🎯 Setting up your personal RAG agent...
🚀 Launching your personal assistant...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d116a1aa9c849fe721.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
